In [1]:
# Lan Le - UB Person Number: 50322056
# CSE555 Problem Set 2: Linear Discriminant Functions and Support Vector Machines




import os
import gzip
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score




def load_mnist(path, kind='train'):

    #Load MNIST data from path
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


# read Digit MNIST dataset
base_dir = os.path.dirname('__file__')
digit_file_path = os.path.join(base_dir, 'data')

X_train, y_train = load_mnist(digit_file_path, kind='train')
X_test, y_test = load_mnist(digit_file_path, kind='t10k')

# scale input data
X_train = X_train/255
X_test = X_test/255

# train Support Vector Classfier
param_grid = {'C': [0.1, 1, 10, 100, 1000]}
clf = LinearSVC(penalty='l1', dual=False, tol=1e-5)
grid = GridSearchCV(clf, param_grid, refit = True, cv = ShuffleSplit())
grid.fit(X_train, y_train)

# make predictions and report performance
y_pred = grid.predict(X_test)
print('Accuracy score: ', accuracy_score(y_test, y_pred))


/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

Accuracy score:  0.9188


/Users/lle/anaconda3/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In order to identify the Lagrange dual problem of the primal problem, we first form the Lagrange function:

\begin{equation}
L = w^T w + C {\sum_{i=1}^{N} {\xi_{j}}} + {\sum_{i=1}^{N} {\alpha_{i} (1-/xi_{i}-y_i w^T x_i)}} + {\sum_{i=1}^{N} \beta_{i} (-\xi_{i})}
\end{equation}

Taking the derivative of the Lagrangian with respect to w and setting it to zero, we have:

\begin{align*}
    \frac{\partial}{\partial w} L = 0  \\
    2w - {\sum_{i=1}^{N} \alpha_{i} y_i x_i} = 0 \\
    w = \frac{1}{2} {\sum_{i=1}^{N} \alpha_{i} y_i x_i}
\end{align*}

Similarly, we take the derivative with respect to $\xi_{i}$ and setting it to zero:

\begin{align*}
    \frac{\partial}{\partial \xi_{i}} L = 0  \\
    C - \alpha_{i} - \beta{i} = 0 \\
    \beta{i} = C - \alpha_{i}
\end{align*}

Since $\beta_{i}$ and $\alpha_{i}$ are greater or equal to 0:

\begin{equation}
0 \leqslant \alpha_{i} \leqslant C
\end{equation}

Substituting $ w = \frac{1}{2} {\sum_{i=1}^{N} \alpha_{i} y_i x_i} $ and $ C = \alpha_{i} + \beta_{i} $ into our original Lagrange function, we have:

\begin{equation}
    L = {\sum_{i=1}^{N} \alpha_{i}} - \frac{3}{4} {\sum_{i,j=1}^{N} \alpha_{i} \alpha_{j} y_i y_j (x_i)^T x_j}
\end{equation}

We obtain the dual problem:

\begin{equation}
    max_{\alpha} L = {\sum_{i=1}^{N} \alpha_{i}} - \frac{3}{4} {\sum_{i,j=1}^{N} \alpha_{i} \alpha_{j} y_i y_j (x_i)^T x_j}
\end{equation}

Subject to $0 \leqslant \alpha_{i} \leqslant C$ for i = 1,...,N.

<br>
Support vectors are the data points that are closest to the decision hyperplane. They are the data points that are the most difficult to classify and therefore, have a direct influence on the position of the decision surface. <br>
<br>
By solving the dual problem, it becomes more efficient for us to classify new point input. With the primal problem, we are able to find the optimal w. However, in order to classify a new point, we have to calculate $w^T x$ which can be computationally expensive.
Meanwhile, with the dual problem, knowing the optimal $\alpha$ we only need to calculate the inner product of the new point and the support vectors (of which there is often only a small number).
In addition, solving the dual problem allows us to apply kernels to our classification problem. <br>
<br>
The margin is $\gamma = \frac{1}{\sqrt{w^{T} w}}$.
The benefits of maximizing the margin: <br>
* Small variations are less likely to affect classification. <br>
* Overfitting is prevented. Thus, the model performs better on test data.

For a problem with K possible classes, we have the following primal problem:
\begin{equation}
min \frac{1}{2} {\sum_{r=1}^{K} {w_r}^T w_r} + C {\sum_{i,r} \xi_{i,r}}
\end{equation}
Subject to ${w_{y_i}}^{T} x_i \geqslant {w_{r}}^{T} x_i + 1 - \xi_{i,r}$ for all i = 1,...,N; $r \neq y_i$ <br>
and $\xi_{i,r} \geqslant 0$